In [1]:
from datasets import load_dataset
from openai import OpenAI
import utils
from tqdm import tqdm
import ast
import utils

import utils
import google.generativeai as genai

genai.configure(api_key=utils.get_GEMINI_API_KEY())
model = genai.GenerativeModel('gemini-pro')

# load gsm8k dataset
gsm8k_dataset = load_dataset("gsm8k", "main")
gsm8k_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [2]:
print(gsm8k_dataset["train"]["answer"][10])

Let S be the number of people on the first hundred years’ ship.
The second hundred years’ ship had twice as many as the first, so it had 2S people.
The third hundred years’ ship had twice as many as the second, so it had 2 * 2S = <<2*2=4>>4S people.
All the ships had S + 2S + 4S = 7S = 847 people.
Thus, the ship that the monster ate in the first hundred years had S = 847 / 7 = <<847/7=121>>121 people on it.
#### 121


### Checker Function

In [3]:
def check(actual, expected):
    """
    Checkes if the actual answer is correct or not

    @param actual:      The actual answer
    @param expected:    The expected answer
    @return:            1 if the actual answer is correct, 0 otherwise
    """
    # Set up the model
    generation_config = {
        "temperature": 0.0,
    }

    prompt = """
        Compare the actual answer with the expected answer. 
        If the actual answer is correct, respond with "1". 
        If the actual answer is incorrect, respond with "0". 
        Respond with a json object with "answer" and "isCorrect" fields.

        Example Prompt:
        Actual answer: The answer is 42.
        Expected answer: 42

        Let's think step by step. 
        The actual answer is 42. The expected answer is also 42. Thus, the actual answer is correct!

        Response: 
        {{
            "answer": "The answer is 42.",
            "isCorrect": 1
        }}

        User Prompt:
        Actual answer: {actual}
        Expected answer: {expected}
    """

    response = model.generate_content(prompt.format(actual=actual, expected=expected.split("####")[-1].strip()), generation_config=generation_config)

    return ast.literal_eval(response.text)["isCorrect"]

def check_fast(actual, expected):
    """
    Faster and simpler version of the check function.
    """
    expected = expected.split("####")[-1].strip()
    return expected in actual

In [4]:
# # Check the check function
# total = 0
# max_tests = 10
# for i in tqdm(range(max_tests)):
#     res = check(gsm8k_dataset["train"]["answer"][i], gsm8k_dataset["train"]["answer"][i])
#     if not res:
#         print(gsm8k_dataset["train"]["answer"][i])
#     total += res

# total/max_tests * 100

### Optimizer

In [5]:
def opt_llm(instruction_score_pairs, training_sample):
    # Format the instruction and score pairs into a string
    pairs_str = ""
    for pair in instruction_score_pairs:
        pairs_str += f"text:\n{pair[0]}\nscore:\n{pair[1]}\n\n"
    
    # Set up the model
    generation_config = {
        "temperature": 1.0,
    }

    prompt = """
        You are an optimization expert. The user has some texts along with their corresponding scores.
        Your task is to generate a new piece of text in square brackets that scores as high as possible.

        I have some texts along with their corresponding scores. The texts are arranged in ascending order
        based on their scores, where higher scores indicate better quality.

        {pairs_str}

        The following exemplars show how to apply your text: you replace <INS> in each input with your
        text, then read the input and give an output. We say your output is wrong if your output is different
        from the given output, and we say your output is correct if they are the same.

        input:
        Q: {q1}
        A: <INS>
        output:
        {a1}

        input:
        Q: {q2}
        A: <INS>
        output:
        {a2}

        input:
        Q: {q3}
        A: <INS>
        output:
        {a3}
        
        Write your new text that is different from the old ones and has a score as high as possible. Write the
        text in square brackets.
    """

    # Passing 20 Best Instruction-Score Pairs
    # 3 randomly chosen questions/answers from the training sample
    q1, q2, q3 = training_sample["question"][0], training_sample["question"][1], training_sample["question"][2]
    a1, a2, a3 = training_sample["answer"][0], training_sample["answer"][1], training_sample["answer"][2]
    response = model.generate_content(prompt.format(pairs_str=pairs_str, q1=q1, q2=q2, q3=q3, a1=a1, a2=a2, a3=a3), 
                                      generation_config=generation_config)
    return response.text[1:-1]

### Scorer

In [6]:
def scorer_lm(instruction, training_sample):
    # Set up the model
    generation_config = {
        "temperature": 0.0,
    }

    # Scoring an instruction using an LM
    accuracy = 0
    prompt = """
        Q: {question}
        A: {instruction}
    """
    for i in tqdm(range(len(training_sample))):
        response = model.generate_content(prompt.format(question=training_sample["question"][i], instruction=instruction), generation_config=generation_config)
        result = response.text
        accuracy += check_fast(result, training_sample["answer"][i])

    accuracy = accuracy / len(training_sample) * 100

    return accuracy

### Combining the Optimizer with the Scorer

In [11]:
# Sample 3.5% of the training set
INS_PER_STEP = 8
EXEMPLARS_PER_STEP = 3
MAX_INS_SCORE_PAIRS = 20
SAMPLE_PERCENTAGE = 0.35/100
alt_percentage = 0.05/100
training_sample = gsm8k_dataset["train"].shuffle(seed=42).select(range(int(len(gsm8k_dataset["train"]) * SAMPLE_PERCENTAGE)))
ins_score_pairs = {"Let's think step by step to solve this problem.": scorer_lm("Let's think step by step to solve this problem.", training_sample)}

steps = 15
for i in tqdm(range(steps)):
    while True:
        try:
            # Optimizer LLM
            exemplars = training_sample.shuffle()[:EXEMPLARS_PER_STEP]
            instructions = [opt_llm(ins_score_pairs, exemplars) for _ in range(INS_PER_STEP)]
            
            # Scoring the new instructions
            new_ins_score_pairs = {ins: scorer_lm(ins, training_sample) for ins in instructions}
            ins_score_pairs.update(new_ins_score_pairs)
            ins_score_pairs = dict(sorted(ins_score_pairs.items(), key=lambda x: x[1], reverse=True)[:MAX_INS_SCORE_PAIRS])
            break
        except ValueError as e:
            print(e)

 88%|████████▊ | 23/26 [02:20<00:18,  6.10s/it]


The `response.text` quick accessor only works for simple (single-`Part`) text responses. This response is not simple text.Use the `result.parts` accessor or the full `result.candidates[index].content.parts` lookup instead.
The `response.text` quick accessor only works for simple (single-`Part`) text responses. This response is not simple text.Use the `result.parts` accessor or the full `result.candidates[index].content.parts` lookup instead.


 38%|███▊      | 10/26 [00:24<00:38,  2.43s/it]


The `response.text` quick accessor only works for simple (single-`Part`) text responses. This response is not simple text.Use the `result.parts` accessor or the full `result.candidates[index].content.parts` lookup instead.


  0%|          | 0/26 [00:04<?, ?it/s]


...

The `response.text` quick accessor only works for simple (single-`Part`) text responses. This response is not simple text.Use the `result.parts` accessor or the full `result.candidates[index].content.parts` lookup instead.


 12%|█▏        | 3/26 [00:23<03:00,  7.86s/it]


The `response.text` quick accessor only works for simple (single-`Part`) text responses. This response is not simple text.Use the `result.parts` accessor or the full `result.candidates[index].content.parts` lookup instead.


  7%|▋         | 1/15 [5:22:16<75:11:46, 19336.17s/it]


InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [ ]:
ins_score_pairs